In [ ]:
using JuMP, Gurobi

In [ ]:
using CSV
data = CSV.read("data_transfered.csv")

In [ ]:
de = 110.25
c = zeros((241,241))
for i = 1:241
    for j = 1:241
        c[i,j] = de*((data[i,8]-data[j,8])^2 + ((data[i,9]-data[j,9])*cos(data[j,8]))^2)^(1/2)
    end
end

In [ ]:
# n stands for the SET of all points in the network
# p stands for the maximum number of distribution point
# assume that all the points could be distribution point
function pmedian(n, p)
    model = Model(Gurobi.Optimizer)
    @variable(model, x[i in n], Bin)
    @variable(model, y[i in n, j in n], Bin)
    @objective(model, Min, sum(c[i, j] * y[i, j] * data[i,10] for i in n, j in n))
    @constraints(model,begin
        [i in n], sum(y[i,j] for j in n) == 1
        [i in n , j in n], y[i,j]-x[j] <= 0
        sum(x[i] for i in n) == p   
        end)
    status = optimize!(model)

    z_opt = JuMP.objective_value(model)
    x_opt = JuMP.value.(x)
    y_opt = JuMP.value.(y)

    return z_opt, x_opt, y_opt
end

In [ ]:
# this function is to add one more distribution center inside the network which already has ONE distribution center
# n stands for the SET of all points in the network
# p_1 stands for the existed distribution center
function plus_1_point(n,p_1)
    model = Model(Gurobi.Optimizer)
    @variable(model,x[i in n],Bin)
    @variable(model,y[i in n, j in n],Bin)
    @objective(model,Min, sum(c[i,j] * y[i,j] * data[i,10] for i in n, j in n))
    @constraints(model,begin
        [i in n], sum(y[i,j] for j in n)==1
        [i in n, j in n], y[i,j] - x[j] <= 0
        sum(x[i] for i in n) == 2
        x[p_1] == 1
        end)
    status = optimize!(model)
    
    z_opt = JuMP.objective_value(model)
    x_opt = JuMP.value.(x)
    y_opt = JuMP.value.(y)
    
    return z_opt, x_opt, y_opt
end

In [ ]:
# this function is to seperate the result from function pmedian(p = 2) and function plus_1_point
# k stands for the result of the function pmedian(p = 2) and function plus_1_point
# n stands for the set of nodes used in the function providing this result
# only 2 distribution center must be existed in the network after running the function pmedian and plus_1_point
function separate(k,n)
    distr=[]
    for i in n
        if k[2][i] > 0.5
            push!(distr,i)
        end
    end
    net_1 = []
    net_2 = []
    for i in n
        if k[3][i,distr[1]] > 0.5
            push!(net_1,i)
        end
    end
    for i in n
        if k[3][i,distr[2]] > 0.5
            push!(net_2,i)
        end
    end
    cost_1 = sum(c[i,j]*data[i,10] for i in net_1, j = distr[1])
    cost_2 = sum(c[i,j]*data[i,10] for i in net_2, j = distr[2])
    return distr[1],net_1,cost_1,distr[2],net_2,cost_2
end
# for the result of this function, element 1 & 4 are the two distribution points, 2 & 5 are the network connected to these two distribution centers, and 3 & 6 are the cost related to these two points